# Welcome to Caustics!

Caustics is a powerful gravitational lensing simulator that can support users from beginner to highly advanced. In this tutorial we will cover the basics of caustics code design and how to get going making your own lensing configurations. If you have a specific lensing goal in mind, there is likely a tutorial which covers enough to get you started!

### Simulating an SIE lens

Here we will demo the very basics of lensing with a classic `SIE` lens model. We will see what it takes to make an `SIE` model, lens a background `Sersic` source, and sample the resulting image using a `Simulator`. Caustics simulators can generalize to very complex scenarios, here we will use a built-in simulator which handles a common use case (lensing a background source). To start, we of course need to import some modules. For the minimal example, this is just `matplotlib` a common package used for plotting, `torch` which is a numerical package for GPU/autodiff (much like `numpy`), and `caustics` the reason you are here.

In [ ]:
import matplotlib.pyplot as plt
import torch
import caustics

### Define a Cosmology

Before we can begin gravitational lensing, we need to know what kind of universe we are in. This is used for calculating various distances and timescales (depending on the problem) since gravitational lensing typically occurs over cosmologically significant distances in the universe. Here we define a standard flat Lambda Cold Dark Matter cosmology. Nothing fancy here, but it's still needed.

In [ ]:
cosmology = caustics.cosmology.FlatLambdaCDM()

### Lens Mass Distribution

In order for gravitational lensing to occur, we need some mass to bend the light. Here we define a basic Singular Isothermal Ellipsoid (SIE), which is a versatile profile used in many strong gravitational lensing simulations. As the first argument, we pass the cosmology so that the `SIE` can compute various quantities which make use of redshift information (seen later). Each model must have a unique name so we call this one `lens` though you can also let Caustics automatically pick a unique name.

In [ ]:
sie = caustics.lenses.SIE(cosmology=cosmology, name="lens")

### Sersic Source Light Distribution

If we wish to see anything in our lensing configuration then we need a bright object in the background to produce some light that will pass through (and be bent by) our lens mass distribution. Here we create a `Sersic` light model which is a common versatile profile for representing galaxies. Note that we don't need to pass a light model any `Cosmology` information, since light models essentially just define a function on `(x,y)` coordinates that gives a brightness, the lens models handle all cosmology related calculations. For the name we very creatively choose `source`.

In [ ]:
src = caustics.light.Sersic(name="source")

### Sersic Lens Light Distribution

The source isn't the only bright thing in the sky! The lensing galaxy itself will also have bright stars and can be seen as well. Let's add another Sersic model with the name `lenslight`.

In [ ]:
lnslt = caustics.light.Sersic(name="lenslight")

### Lens Source Simulator

Next we pass our configuration to a `Simulator` in caustics, simulators perform the work of forward modelling various configurations and producing the desired outputs. Here we are interested in a common scenario of producing an image of a background source through a lens distribution. It is possible to make your own simulator to represent all sorts of situations. First, we pass the `lens` model and the `source` model defined above. Next we use `pixelscale` and `pixels_x` to define the grid of pixels that will be sampled. Finally, we pass the `z_s` redshift at which the source (`Sersic`) model should be placed; recall that light models don't use the cosmology model and so aren't aware of their placement in space.

In [ ]:
minisim = caustics.sims.Lens_Source(
    lens=sie, source=src, lens_light=lnslt, pixelscale=0.05, pixels_x=100
)

### Model Parameters

Each of the `lens`, `source`, and `lenslight` models have their own parameters that are needed to sample a given lensing configuration. There are a number of ways to pass these parameters to a Caustics simulator, but the most straightforward for most purposes is as a Pytorch `Tensor`. In this cell, we build the tensor manually for our configuration. In general one would have a script generate the configuration, or an optimizer/sampler would randomly choose parameter values.

In order, here is an explanation of the parameters. `z_s` is the redshift of the source. `z_l` is the lens redshift which tells the lens how far away it is from the observer (us). The next two parameters `x0` and `y0` indicate where the lens is relative to the main optical axis, which is the coordinates `(0, 0)`. The `q` parameter gives the axis ratio for the `SIE`, so it knows how elongated it is. Then `phi` indicates the position angle (where the ellipse is pointing). `b` gives the Einstein radius (in arcsec) of the lens. The next `x0` and `y0` provide the position relative to the main optical axis of the Sersic source, here we offset the source slightly to make for an interesting figure. The `q` parameter defines the axis ratio of the Sersic ellipse. `phi` defines the position angle of the ellipse. `n` is the Sersic index which determines how concentrated the light is; `n=0.5` is a Gaussian distribution, `n=1` is an exponential, `n=4` is a De Vaucouleurs profile. `Re` is the radius (in arcsec) within which half the total light of the profile is enclosed. `Ie` is the brightness at `Re`. The next set of parameters are also Sersic parameters, but this time they are for the lens light model.

In [ ]:
x = torch.tensor([
#   z_s  z_l   x0   y0   q    phi     b    x0   y0   q     phi    n    Re
    1.5, 0.5, -0.2, 0.0, 0.4, 1.5708, 1.7, 0.0, 0.0, 0.5, -0.985, 1.3, 1.0, 
#   Ie    x0   y0   q    phi  n   Re   Ie
    5.0, -0.2, 0.0, 0.8, 0.0, 1., 1.0, 10.0
])  # fmt: skip

### Plot the Results!

This section is mostly self explanitory. We evaluate the simulator configuration by calling it like a function. There are several possible arguments to the simulator function, here we use `quad_level` which indicates the depth of quadrature integration to use for each pixel (how accurate to make the flux).

In [ ]:
image = minisim(x, quad_level=3).detach().cpu().numpy()

plt.imshow(image, origin="lower")
plt.axis("off")
plt.show()

### The Simulator Graph

Here we take a quick look at the simulator graph for the image we have produced. You will learn much more about what this means in the `Simulators` tutorial notebook, but let's cover the basics here. First, note that this is a Directed Acyclic Graph (DAG), this is how all simulator parameters are represented in Caustics. At the top of the graph is the `Lens_Source` object, you can see in brackets it has a name `sim` which is used as the identifier for it's node in the graph. At the next level is the `z_s` parameter for the redshift of the source. Next are the `SIE` lens, `Sersic` source, and `Sersic` lenslight objects which themselves each hold parameters. You will notice that all the parameters are in white boxes right now, this is because they are `dynamic` parameters which need values to be passed, grey boxes are used for parameters with fixed values.

In [ ]:
minisim.get_graph(True, True)

### Sampling with a Simulator

Now let's see how to use some of the powerful features of the simulator we have created. Note that it behaves essentially like a function, allowing us to take advantage of many PyTorch features. To start, lets see how we can run batches of lens simulations using `vmap`.

In [ ]:
# Make 5 copies of the lens parameters
newx = x.repeat(20, 1)
# Randomly modify the lens parameters
newx += torch.normal(mean=0, std=0.1 * torch.ones_like(newx))

# Sample the new images
images = torch.vmap(minisim)(newx)

# Plot the new images
fig, axarr = plt.subplots(4, 5, figsize=(20, 16))
for ax, im in zip(axarr.flatten(), images):
    ax.imshow(im, origin="lower")
    ax.axis("off")
plt.tight_layout()
plt.savefig("minisim_vmap.png", transparent=True)
plt.show()

### Gradients with autodiff

Batching is useful for fully paramellizing code and maximally using computational resources, but autodiff gradients allow whole new algorithms and techniques to be used in gravitational lensing! Let's try computing the Jacobian for the lensing configuration that we have been using so far. The result is a grid of images that show how the lensing simulation image would change if we adjusted each parameter individually. Thanks to autodiff, these derivatives have no finite differences approximation error, they are exact up to the machine precision.

In [ ]:
J = torch.func.jacfwd(minisim)(x)

# Plot the new images
fig, axarr = plt.subplots(4, 5, figsize=(20, 16))
for i, ax in enumerate(axarr.flatten()):
    ax.imshow(J[..., i + 1], origin="lower")
    ax.axis("off")
plt.tight_layout()
plt.savefig("minisim_jacobian.png", transparent=True)
plt.show()

## Where to go next?

The caustic tutorials are generally short and to the point, that way you can identify what you want and jump right to some useful code that demo's the particular problem you face. Below is a list of caustic tutorials and a quick description of what you will learn in each one::

- `LensZoo`: here you can see all the built-in lens mass distributions in `caustic` and how they distort the same background Seric source.
- `Playground`: here we demo the main visualizations of a lensing system (deflection angles, convergence, potential, time delay, magnification) in an interactive display so you can change the parameters by hand and see how the visuals change!
- `VisualizeCaustics`: here you can see how to find and display caustics, a must when using `caustic`!
- `Simulators`: here we describe the powerful simulator framework and how it can be used to quickly swap models, parameters, and other features and turn a complex forward model into a simple function.
- `InvertLensEquation`: here we demo forward ray tracing in `caustic` the process of mapping from the source plane to the image plane.